In [1]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository



# Model Module

* Create a `SQLRepository` named `repo`. Be sure that it's attached to a SQLite connection.


In [2]:
connection = sqlite3.connect(database=settings.db_name, check_same_thread=False)
repo = SQLRepository(connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


* In the `model` module, create a definition for a `GarchModel` model class. For now, it should only have an `__init__` method. Use the docstring as a guide. When you're done, test your class using the assert statements below.


In [4]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

* Write a `wrangle_data` method for your `GarchModel` class. When you're done, use the assert statements below to test the method by getting and wrangling data for the department store [Shoppers Stop](https://www.shoppersstop.com/).


In [10]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2019-05-10    0.580758
2019-05-13   -0.719033
2019-05-14    0.801053
2019-05-15   -1.012410
2019-05-16   -1.077752
Name: return, dtype: float64

* Create a `fit` method for your `GarchModel` class. When you're done, use the code below to test it.


In [37]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2430.56
Distribution:                  Normal   AIC:                           4869.12
Method:            Maximum Likelihood   BIC:                           4888.75
                                        No. Observations:                 1000
Date:                Tue, May 23 2023   Df Residuals:                      999
Time:                        17:04:33   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1479  7.621e-02      1.940  5.234e-02 [-1.494e-03,  0.297]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.1520      0.133      1.145      0.252     [ -0.108,  0.412]
alpha[1]       0.0366  1.763e-02      2.075  3.800e-02 [2.025e-03,7.112e-02]
beta[1]        0.9467  2.922e-02     32.398 2.923e-230     [  0.889,  1.004]
============================================================================

Covariance estimator: robust
"""

* Create a `predict_volatility` method for your `GarchModel` class. Your method will need to return predictions as a dictionary, so you'll need to add your `clean_prediction` function as a helper method. When you're done, test your work using the assert statements below.


In [14]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-05-23T00:00:00': 2.2561943178707318,
 '2023-05-24T00:00:00': 2.2710221665746224,
 '2023-05-25T00:00:00': 2.2855089413955323,
 '2023-05-26T00:00:00': 2.2996651469642324,
 '2023-05-29T00:00:00': 2.31350081877044}

* Create a `dump` method for your `GarchModel` class. It should save the model assigned to the `model` attribute to the folder specified in your configuration `settings`. Use the docstring as a guide, and then test your work below.

In [18]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models/2023-05-23T16:54:16.886223_SHOPERSTOP.BSE.pkl'

* Create a `load` function below that will take a ticker symbol as input and return a model. When you're done, use the next cell to load the Shoppers Stop model you saved.


In [29]:
def load(ticker):

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")

    # Try to find path of latest model
    try:
        model_path = sorted(glob(pattern))[-1]
        print(model_path)
    # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No model with {ticker}.")

    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

In [33]:
# Assign load output to `model`
model_shop = load(ticker="SHOPERSTOP.BSE")

# Does function return an `ARCHModelResult`
assert isinstance(model_shop, ARCHModelResult)

# Check model parameters
model_shop.summary()

models/2023-05-23T16:54:16.886223_SHOPERSTOP.BSE.pkl


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2430.56
Distribution:                  Normal   AIC:                           4869.12
Method:            Maximum Likelihood   BIC:                           4888.75
                                        No. Observations:                 1000
Date:                Tue, May 23 2023   Df Residuals:                      999
Time:                        16:27:32   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1479  7.621e-02      1.940  5.234e-02 [-1.494e-03,  0.297]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.1520      0.133      1.145      0.252     [ -0.108,  0.412]
alpha[1]       0.0366  1.763e-02      2.075  3.800e-02 [2.025e-03,7.112e-02]
beta[1]        0.9467  2.922e-02     32.398 2.923e-230     [  0.889,  1.004]
============================================================================

Covariance estimator: robust
"""

* Transform your `load` function into a method for your `GarchModel` class. When you're done, test the method using the assert statements below.


In [5]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")

model_shop.model.summary()

models/2023-05-23T16:54:16.886223_SHOPERSTOP.BSE.pkl


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2430.56
Distribution:                  Normal   AIC:                           4869.12
Method:            Maximum Likelihood   BIC:                           4888.75
                                        No. Observations:                 1000
Date:                Tue, May 23 2023   Df Residuals:                      999
Time:                        16:27:32   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1479  7.621e-02      1.940  5.234e-02 [-1.494e-03,  0.297]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.1520      0.133      1.145      0.252     [ -0.108,  0.412]
alpha[1]       0.0366  1.763e-02      2.075  3.800e-02 [2.025e-03,7.112e-02]
beta[1]        0.9467  2.922e-02     32.398 2.923e-230     [  0.889,  1.004]
============================================================================

Covariance estimator: robust
"""

Our `model` module is done! Now it's time to move on to the "main" course and add the final piece to our application.

# Main Module

* In the `main` module, instantiate a FastAPI application named `app`.



* Go to the command line, navigate to the directory for this project, and start your app server by entering the following command.

```bash
uvicorn main:app --reload --workers 1 --host localhost --port 8008
```

Remember how the AlphaVantage API had a `"/query"` path that we accessed using a `get` HTTP request? We're going to build similar paths for our application. Let's start with an MVP example so we can learn how paths work in FastAPI. 

* Create a `"/hello"` path for your app that returns a greeting when it receives a `get` request.



We've got our path. Let's perform as `get` request to see if it works.

* Create a `get` request to hit the `"/hello"` path running at `"http://localhost:8008"`.


In [10]:
url = "http://localhost:8008/hello"
response = requests.get(url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello :)'}

## `"/fit"` Path

Our first path will allow the user to fit a model to stock data when they make a `post` request to our server. They'll have the choice to use new data from AlphaVantage, or older data that's already in our database. When a user makes a request, they'll receive a response telling them if the operation was successful or whether there was an error. 

One thing that's very important when building an API is making sure the user passes the correct parameters into the app. Otherwise, our app could crash! FastAPI works well with the [pydantic library](https://pydantic-docs.helpmanual.io/), which checks that each request has the correct parameters and data types. It does this by using special data classes that we need to define. Our `"/fit"` path will take user input and then output a response, so we need two classes: one for input and one for output.

* Create definitions for a `FitIn` and a `FitOut` data class. The `FitIn` class should inherit from the pydantic `BaseClass`, and the `FitOut` class should inherit from the `FitIn` class. Be sure to include type hints.


With our data classes defined, let's see how pydantic ensures our that users are supplying the correct input and our application is returning the correct output.

* Use the code below to experiment with your `FitIn` and `FitOut` classes. Under what circumstances does instantiating them throw errors? What class or classes are they instances of?


In [17]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker="SHOPERSTOP.BSE",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker="SHOPERSTOP.BSE",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1,
    success=True,
    message="Ready"
)
print(fo)

ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1
ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1 success=True message='Ready'


* Create a `build_model` function in your `main` module. Use the docstring as a guide, and test your function below.


In [20]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="SHOPERSTOP.BSE", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_shop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_shop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_shop.ticker == "SHOPERSTOP.BSE"

# Is `use_new_data` attribute correct?
assert not model_shop.use_new_data

model_shop

We've got data classes, we've got a `build_model` function, and all that's left is to build the `"/fit"` path. We'll use our `"/hello"` path as a template, but we'll need to include more features, like error handling. 

* Create a `"/fit"` path for your `app`. It will take a `FitIn` object as input, and then build a `GarchModel` using the `build_model` function. The model will wrangle the needed data, fit to the data, and save the completed model. Finally, it will send a response in the form of a `FitOut` object. Be sure to handle any errors that may arise. 



Last step! Let's make a `post` request and see how our app responds.

* Create a `post` request to hit the `"/fit"` path running at `"http://localhost:8008"`. You should train a GARCH(1,1) model on 2000 observations of the Shoppers Stop data you already downloaded. Pass in your parameters as a dictionary using the `json` argument.


In [30]:
# URL of `/fit` path
url =  "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker": "SHOPERSTOP.BSE",
    "use_new_data": False,
    "n_observations": 2000,
    "p": 1,
    "q": 1
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'SHOPERSTOP.BSE',
 'use_new_data': False,
 'n_observations': 2000,
 'p': 1,
 'q': 1,
 'success': True,
 'message': 'Trained and saved models/2023-05-23T20:47:53.691047_SHOPERSTOP.BSE.pkl'}

## `"/predict"` Path

For our `"/predict"` path, users will be able to make a `post` request with the ticker symbol they want a prediction for and the number of days they want to forecast into the future. Our app will return a forecast or, if there's an error, a message explaining the problem.

The setup will be very similar to our `"/fit"` path. We'll start with data classes for the in- and output.

* Create definitions for a `PredictIn` and `PredictOut` data class. The `PredictIn` class should inherit from the pydantic `BaseModel`, and the `PredictOut` class should inherit from the `PredictIn` class. Be sure to include type hints. The use the code below to test your classes.


In [32]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="SHOPERSTOP.BSE", n_days=5)
print(pi)

po = PredictOut(
    ticker="SHOPERSTOP.BSE", n_days=5, success=True, forecast={}, message="success"
)
print(po)

ticker='SHOPERSTOP.BSE' n_days=5
ticker='SHOPERSTOP.BSE' n_days=5 success=True forecast={} message='success'


* Create a `"/predict"` path for your `app`. It will take a `PredictIn` object as input, build a `GarchModel`, load the most recent trained model for the given ticker, and generate a dictionary of predictions. It will then return a `PredictOut` object with the predictions included. Be sure to handle any errors that may arise.


* Create a `post` request to hit the `"/predict"` path running at `"http://localhost:8008"`. You should get the 5-day volatility forecast for Shoppers Stop. When you're satisfied, submit your work to the grader.


In [44]:
# URL of `/predict` path
url = "http://localhost:8080/predict"
# Data to send to path
json = {
    "ticker": "SHOPERSTOP.BSE",
    "n_days": 5
}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

{'ticker': 'SHOPERSTOP.BSE',
 'n_days': 5,
 'success': True,
 'forecast': {'2023-05-23T00:00:00': 2.0578735128364825,
  '2023-05-24T00:00:00': 2.0699589780160834,
  '2023-05-25T00:00:00': 2.081761772547934,
  '2023-05-26T00:00:00': 2.0932904268523296,
  '2023-05-29T00:00:00': 2.1045531089047587},
 'message': ''}